In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import figure
import random
from random import randint

print('Setup Complete')

In [ ]:
#Define folder path (Custom)
#Đường dẫn đến nơi lưu file .csv của Predict => modify lại theo đường dẫn của tị em (nhớ Add data)
RESULT_DIR = '/kaggle/input/bmseresult-tempo/result_vin.csv'

#CSV file include width and height
#Đường dẫn đến nới lưu file .csv (bao gồm width và height) => modify lại theo đường dẫn của tị em (nhớ Add data)
IMAGE_DIR = '/kaggle/input/vinbigdata-640pixel/train_vin.csv'
# ===============================
df = pd.read_csv(RESULT_DIR)
display(df.head())
# ===============================
df_wh = pd.read_csv(IMAGE_DIR)
df_wh.fillna(0, inplace=True)
df_wh = df_wh[df_wh.class_id!=14].reset_index(drop = True)
display(df_wh.head())
# ===============================
#879 images id
image_ids = df.image_id.unique()
print(len(image_ids))

In [ ]:
# map label_id to specify color
class_ids = df_wh['class_id'].unique()
label2color = {class_id:[randint(0,255) for i in range(3)] for class_id in class_ids}
thickness = 3

In [ ]:
# ===============================
# Có 2 option:
# Option 1: Mỗi lần Run sẽ hiển thị random một ảnh trong số 879 ảnh test
# Option 2: Mỗi lần Run sẽ hiển thị một ảnh trong số 879 ảnh test (Các em chỉnh index lần lượt từ 0-878 để lướt qua cả 879 ảnh)
#Option 3: Vẽ theo id cụ thể, tụi em thay thế id ảnh vào
# Tất cả đều kèm theo ID ảnh
# Tụi em chạy lần lượt 879 ảnh để tìm tầm 10-20 cái ảnh có tỷ lệ dự đoán đẹp nhất mà cho vào report
# ===============================


#ID and path
#Option 1: Bỏ '#' để chọn
img_id = random.choice(image_ids)
#Option 1: Bỏ '#' để chọn
#img_id = image_ids[0]
#Option 3: Bỏ '#' để chọn
#img_id = 'Dán id ảnh vào đây'

#Đường dẫn đến nơi lưu ảnh 640 => modify lại theo đường dẫn của tị em (nhớ Add data)
img_path = f'/kaggle/input/vinbigdata-640pixel/Train/{img_id}.jpg'

#Draw Origin
scale = df_wh[df_wh['image_id'] == img_id].height.values[1]/640
image = cv2.imread(img_path)
boxes = df_wh.loc[df_wh['image_id'] == img_id, ['x_min', 'y_min', 'x_max', 'y_max']].values/scale
labels = df_wh.loc[df_wh['image_id'] == img_id, ['class_id']].values.squeeze()

for label_id, box in zip(labels, boxes):
    color = label2color[label_id]
    image = cv2.rectangle(
        image,
        (int(box[0]), int(box[1])),
        (int(box[2]), int(box[3])),
        color, thickness
    )  

#Draw Predict
image_prd = cv2.imread(img_path)
boxes_prd = df.loc[df['image_id'] == img_id, ['x_min', 'y_min', 'x_max', 'y_max']].values*640
labels_prd = df.loc[df['image_id'] == img_id, ['class_id']].values.squeeze()

for label_id, box in zip(labels_prd, boxes_prd):
    color = label2color[label_id]
    image_prd = cv2.rectangle(
        image_prd,
        (int(box[0]), int(box[1])),
        (int(box[2]), int(box[3])),
        color, thickness
    )  


#Subplot
fig = figure(figsize=(20, 20), dpi=80)
fig.add_subplot(1, 2, 1)
plt.imshow(image)
fig.add_subplot(1, 2, 2)
plt.imshow(image_prd)


print(f'Image Id: {img_id}')
print('Number of origin label:')
print(len(labels))
print('Number of predict label:')
print(len(labels_prd))
